# Analyzing eBay Kleinanzeigen data
In this project we'll be looking at a sample of 50.000 data points from the original eBay Kleinanzeigen dataset. The full data set can be found __[here.](https://data.world/data-society/used-cars-data)__

Our goal is to clean up the data and then run a few analysis to get a better picture of the used car market of that time period.

## 1. Getting to know the data

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

autos = pd.read_csv("autos.csv",
                    encoding="Latin-1"
                   )

In [25]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [26]:
autos.info()
autos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


### Key observations:

For a number of cars we are missing the info about:

- Vehicle type (10.2%)
- The transmission type (5.3%)
- The model name (5.5%)
- The type of fuel the cars use (8.9%)
- Whether the car has damage or not (can possibly be explained by the fact that not all cars sold had damages) (19.7%)

#### Let's rename the headers so it's easier to use them later on:

In [27]:
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

In [28]:
autos.rename(columns={'yearOfRegistration' : 'registration_year',
                     'monthOfRegistration':'registration_month',
                     'notRepairedDamage':'unrepaired_damage',
                     'dateCreated':'ad_created',
                     'nrOfPictures':'pictures',
                     'postalCode':'postal_code',
                     'offerType':'offer_type',
                     'vehicleType':'vehicle_type',
                     'fuelType':'fuel_type',
                     'lastSeen':'last_seen_online',
                     'dateCrawled':'date_crawled',
                     'abtest':'AB_test'},inplace=True)

## 2. Data exploration and cleaning
Let's have a closer look at the data to see where we might need to clean it.

In [29]:
autos.describe(include="all")

,date_crawled,name,seller,offer_type,price,AB_test,vehicle_type,registration_year,gearbox,powerPS,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,pictures,postal_code,last_seen_online
count,50000,50000,50000,50000,50000,50000,44905,50000.00,47320,50000.00,47242,50000,50000.00,45518,50000,40171,50000,50000.00,50000.00,50000
unique,48213,38754,2,2,2357,2,8,nan,2,nan,245,13,nan,7,40,2,76,nan,nan,39481
top,2016-03-29 23:42:13,Ford_Fiesta,privat,Angebot,$0,test,limousine,nan,manuell,nan,golf,"150,000km",nan,benzin,volkswagen,nein,2016-04-03 00:00:00,nan,nan,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,nan,36993,nan,4024,32424,nan,30107,10687,35232,1946,nan,nan,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.07,NaN,116.36,NaN,NaN,5.72,NaN,NaN,NaN,NaN,0.00,50813.63,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.71,NaN,209.22,NaN,NaN,3.71,NaN,NaN,NaN,NaN,0.00,25779.75,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.00,NaN,0.00,NaN,NaN,0.00,NaN,NaN,NaN,NaN,0.00,1067.00,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.00,NaN,70.00,NaN,NaN,3.00,NaN,NaN,NaN,NaN,0.00,30451.00,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.00,NaN,105.00,NaN,NaN,6.00,NaN,NaN,NaN,NaN,0.00,49577.00,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.00,NaN,150.00,NaN,NaN,9.00,NaN,NaN,NaN,NaN,0.00,71540.00,NaN


#### This uncovers a few things:

1. The coloumns **price** and **odometer** are numeric values stored as strings. We'll need to convert them.
2. The **seller** and **offer_type** columns contain the same values in all but one entry. This makes them redundant for our analysis as they don't add much information about the listings.
3. The **registration_year** column has some extreme values. We'll delete them from our data.

### 2.1 Cleaning price and odometer columns

Just by looking at the values above, we know that we'll have to remove '$' as well as 'km' from the two columns. However, we don't know from the data above if the price is written with "," or "." or just a space to indicate a number. 

Let's check that first:

In [30]:
autos["price"].head(5)

0    $5,000
1    $8,500
2    $8,990
3    $4,350
4    $1,350
Name: price, dtype: object

We see that a comma is used to separate numbers above $999. We must therefore exclude that character from the column before we can covert the data to numeric type.

In [31]:
autos["price"] = autos["price"].str.replace('$','').str.replace(',','').astype(int)


Let's clean the **odometer** columns next. 

We already know that we need to remove the 'km' but what about any potential commas or periods?

In [32]:
autos["odometer"].head()

0    150,000km
1    150,000km
2     70,000km
3     70,000km
4    150,000km
Name: odometer, dtype: object

A comma is used to separate the thousands, so we'll remove it as well before we convert the data to numeric type:

In [33]:
autos["odometer"] = autos["odometer"].str.replace('km','').str.replace(',','').astype(int)


**Now that both price and odometer columns are numeric, we can check for outliers**

We'll start with the price column.

In [34]:
autos["price"].unique().shape

(2357,)

In [35]:
autos["price"].describe()

count      50000.00
mean        9840.04
std       481104.38
min            0.00
25%         1100.00
50%         2950.00
75%         7200.00
max     99999999.00
Name: price, dtype: float64

#### Observations:

- There are 2357 unique prices in the data set
- The average price of a car is 9 840 USD. 
- The cheapest car is actually free, priced at 0. 
- The most expensive car seems to be prices at 100 million. That might be an error. 

We'll need to look closer at really big prices so that our analysis isn't skewed down the line:

In [36]:
autos["price"].value_counts().sort_index(ascending=False).head(20)

99999999    1
27322222    1
12345678    3
11111111    2
10000000    1
3890000     1
1300000     1
1234566     1
999999      2
999990      1
350000      1
345000      1
299000      1
295000      1
265000      1
259000      1
250000      1
220000      1
198000      1
197000      1
Name: price, dtype: int64

There is a big jump in price after 350 000 USD. Some of the prices might be a random input - such as 99999999 and 12345678. Overall, all of the prices above 350 000 USD seem very high for a used car. 

Let's see the model of the cars to see if the prices makes sense.

In [37]:
price_filter = (autos["price"] > 350000)
strange_price = autos[price_filter]
strange_price.loc[:,["name","price","model"]].sort_values(by=["price"],ascending=False)

,name,price,model
39705,Tausch_gegen_gleichwertiges,99999999,s_klasse
42221,Leasinguebernahme,27322222,c4
27371,Fiat_Punto,12345678,punto
39377,Tausche_volvo_v40_gegen_van,12345678,v40
47598,Opel_Vectra_B_1_6i_16V_Facelift_Tuning_Showcar...,12345678,vectra
2897,Escort_MK_1_Hundeknochen_zum_umbauen_auf_RS_2000,11111111,escort
24384,Schlachte_Golf_3_gt_tdi,11111111,NaN
11137,suche_maserati_3200_gt_Zustand_unwichtig_laufe...,10000000,NaN
47634,Ferrari_FXX,3890000,NaN
7814,Ferrari_F40,1300000,NaN


Only a few of the entries with high prices are actually luxury cars. It is therefore more plausible that they can be sold for such high prices. Therefore, we can assumes that only the Maserati and the two Ferraris are legitamate listings.

However, the three cars are much more expensive than the average price. Given that we are dealing with data of non-luxury brands, these three entires might again skew our analysis without adding too much value.

We will therefore remove them as well.

In [38]:
cars_drop = autos["price"] > 350000
autos = autos.drop(autos.index[cars_drop])

### Let us look into the column odometer next


In [39]:
autos["odometer"].describe()

count    49986.00
mean    125736.51
std      40038.13
min       5000.00
25%     125000.00
50%     150000.00
75%     150000.00
max     150000.00
Name: odometer, dtype: float64

#### Observations:

- The average amount of km that a used car has driven before sale is 125 732 km. 
- By looking at the 25% and 75% percentile, we see that a lot of cars have a mileage of 125 000 km or higher.

In [40]:
autos["odometer"].value_counts().sort_index(ascending=False).head(20)

150000    32416
125000     5169
100000     2168
90000      1757
80000      1436
70000      1230
60000      1164
50000      1025
40000       818
30000       789
20000       784
10000       264
5000        966
Name: odometer, dtype: int64

Indeed, almost 65% of all cars for sale has a mileage of 150 000 km. And almost 80% of all cars for sale have mileage over 100.000 km. 

None of the values stick out so we won't be removing any.

### 2.2 Exploring and cleaning date related columns

There are 4 columns that reporesent dates:

- date_crawled: when the ad was crawled
- last_seen_online: when the ad was last seen online
- ad_created: when the car was listed for sale
- registration_month
- registration_year

In [41]:
autos["date_crawled"].describe()

count                   49986
unique                  48200
top       2016-04-04 16:40:33
freq                        3
Name: date_crawled, dtype: object

In [42]:
autos["date_crawled"].value_counts().sort_index()

2016-03-05 14:06:30    1
2016-03-05 14:06:40    1
2016-03-05 14:07:04    1
2016-03-05 14:07:08    1
2016-03-05 14:07:21    1
                      ..
2016-04-07 14:30:09    1
2016-04-07 14:30:26    1
2016-04-07 14:36:44    1
2016-04-07 14:36:55    1
2016-04-07 14:36:56    1
Name: date_crawled, Length: 48200, dtype: int64

In [50]:
(autos["date_crawled"]
 .str[:10]
 .value_counts()
 .sort_index()
)

2016-03-05    1269
2016-03-06     697
2016-03-07    1798
2016-03-08    1663
2016-03-09    1660
2016-03-10    1606
2016-03-11    1624
2016-03-12    1838
2016-03-13     778
2016-03-14    1831
2016-03-15    1699
2016-03-16    1475
2016-03-17    1575
2016-03-18     653
2016-03-19    1745
2016-03-20    1891
2016-03-21    1874
2016-03-22    1645
2016-03-23    1619
2016-03-24    1455
2016-03-25    1587
2016-03-26    1624
2016-03-27    1552
2016-03-28    1742
2016-03-29    1707
2016-03-30    1681
2016-03-31    1595
2016-04-01    1690
2016-04-02    1770
2016-04-03    1934
2016-04-04    1824
2016-04-05     655
2016-04-06     159
2016-04-07      71
Name: date_crawled, dtype: int64

In [44]:
(autos["date_crawled"]
 .str[5:7]
 .value_counts(normalize=True,dropna=False)
 .sort_index()
)

03   0.84
04   0.16
Name: date_crawled, dtype: float64

Last_seen_online could be the date when an ad was taking down, which can give us an indication of how long it takes to sell a car.

In [52]:
(autos["last_seen_online"]
 .str[:10]
 .value_counts()
 .sort_index()
)

2016-03-05       54
2016-03-06      221
2016-03-07      268
2016-03-08      379
2016-03-09      492
2016-03-10      538
2016-03-11      626
2016-03-12     1190
2016-03-13      449
2016-03-14      640
2016-03-15      794
2016-03-16      822
2016-03-17     1396
2016-03-18      371
2016-03-19      787
2016-03-20     1035
2016-03-21     1036
2016-03-22     1079
2016-03-23      929
2016-03-24      978
2016-03-25      960
2016-03-26      848
2016-03-27      801
2016-03-28     1042
2016-03-29     1116
2016-03-30     1242
2016-03-31     1191
2016-04-01     1155
2016-04-02     1244
2016-04-03     1268
2016-04-04     1231
2016-04-05     6212
2016-04-06    11046
2016-04-07     6546
Name: last_seen_online, dtype: int64